In [1]:
import pandas as pd
import pickle
from scipy import stats
import pandas as pd
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s
import numpy as np

c:\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
c:\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [14]:
# import data from main experimental period
df = pickle.load(open( '../00_data/cleaned/sitzungssummary.pkl','rb') )
xmlsaetze = pickle.load(open( '../00_data/cleaned/xmlsaetze.pkl','rb') )
intv = df[['UserID','interventiongroup']]

# import data from additional experimental period
df_nov_mar = pickle.load(open( '../00_data/competency_development/sitzungssummary_nov_mar.pkl','rb') )
xmlsaetze_nov_mar = pickle.load(open( '../00_data/competency_development/xmlsaetze_nov_mar.pkl','rb') )
intv_nov_mar = df_nov_mar[['UserID','interventiongroup']]

df = pd.concat([df,df_nov_mar])
xmlsaetze = pd.concat([xmlsaetze, xmlsaetze_nov_mar])

In [3]:
# get a list of user that had sessions in july and in late september / october
df_min = df.groupby(['UserID','UebungsID']).agg({'Datum':np.min})
df_max= df.groupby(['UserID','UebungsID']).agg({'Datum':np.max})
df_min = df_min[(df_min['Datum'] > '2022-06-21') & (df_min['Datum'] < '2022-09-21')]
df_max = df_max[(df_max['Datum'] > '2022-11-21') & (df_max['Datum'] < '2023-02-21')]
merge = pd.merge(df_min, df_max, on='UserID',how='inner')
merge = merge.reset_index()
example_user = merge[['UserID']]
example_user = example_user.drop_duplicates()

In [5]:
# ability für die personen mit items in zeitraum 1 berechnen
# ability für die personen mit items in zeitraum 2 berechnen = gesamter zeitraum
# vergleichen 

Time period 1

In [14]:
rasch_1 = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]
rasch_1['UserID'].astype(str)
rasch_1['SatzID'].astype(str)

#es werden nur user aus der liste example user behalten
rasch_1 = pd.merge(example_user, rasch_1, on='UserID', how='left')

# only use input from the first time
rasch_1 = rasch_1[(rasch_1['Datum'] > '2022-06-21') & (rasch_1['Datum'] < '2022-09-21')]

In [15]:
test = rasch_1.groupby(['UserID','SatzID']).agg({'Datum':np.max}) # hat nichts mit den users zu tun, holt nur alle satz ids
test = test.reset_index()
test = pd.merge(test, rasch_1, on=['UserID','SatzID','Datum'])
rasch_1 = test.pivot_table(values='Erfolg', index=test.UserID, columns='SatzID', aggfunc='last')

In [36]:
delete_li = []
for col in rasch_1:
    cnt = rasch_1[col].count()
    if cnt < 2:
        delete_li.append(col)

In [37]:
delete_li

[]

In [18]:
rasch_1 = rasch_1.drop(columns=[9991889.0, 9991891.0, 9991895.0, 9992088.0, 9992089.0, 9992094.0, 9992110.0])

In [24]:
rasch_1 = rasch_1.drop(columns=[127.0 ,147.0 ,229.0 ,233.0 ,234.0 ,236.0 ,245.0 ,246.0 ,247.0 ,248.0 ,249.0 ,253.0 ,254.0 ,255.0 ,259.0 ,264.0 ,265.0 ,271.0 ,272.0 ,275.0 ,276.0 ,373.0 ,374.0 ,375.0 ,376.0 ,377.0 ,378.0 ,379.0 ,380.0 ,381.0 ,382.0 ,383.0 ,384.0 ,385.0 ,386.0 ,387.0 ,391.0 ,392.0 ,394.0 ,395.0 ,396.0 ,397.0 ,400.0 ,583.0 ,584.0 ,653.0 ,656.0 ,658.0 ,661.0 ,665.0 ,667.0 ,668.0 ,669.0 ,670.0 ,781.0 ,783.0 ,785.0 ,800.0 ,808.0 ,809.0 ,818.0 ,820.0 ,822.0 ,828.0 ,873.0 ,887.0 ,891.0 ,1696.0 ,1706.0 ,1709.0 ,1710.0 ,1711.0 ,1714.0 ,1719.0 ,1720.0 ,2686.0 ,2687.0 ,2691.0 ,2694.0 ,2705.0 ,2709.0 ,2710.0 ,2711.0 ,3893.0 ,3894.0 ,3895.0 ,3896.0 ,3897.0 ,3898.0 ,3899.0 ,3900.0 ,3901.0 ,3902.0 ,3903.0 ,3904.0 ,3905.0 ,3906.0 ,3907.0 ,3908.0 ,3909.0 ,3910.0 ,3911.0 ,3912.0 ,3913.0 ,3914.0 ,3915.0 ,3916.0 ,3917.0 ,3918.0 ,3919.0 ,3920.0 ,3921.0 ,3922.0 ,3924.0 ,3926.0 ,3927.0 ,3928.0 ,3929.0 ,3930.0 ,3931.0 ,3932.0 ,3933.0 ,3934.0 ,3935.0 ,3936.0])

In [30]:
rasch_1 = rasch_1.drop(columns=[252.0 ,1707.0 ,3937.0 ,3938.0 ,3939.0 ,3940.0 ,3941.0 ,3942.0 ,3943.0 ,3944.0 ,3945.0 ,3946.0 ,3947.0 ,3948.0 ,3949.0 ,3950.0 ,3951.0 ,3952.0 ,3957.0 ,3958.0 ,3959.0 ,3961.0 ,3962.0 ,3963.0 ,3964.0 ,3965.0 ,3967.0 ,3973.0 ,3974.0 ,3975.0 ,3976.0 ,3978.0 ,3979.0 ,3980.0 ,3981.0 ,3982.0 ,4073.0 ,4074.0 ,4075.0 ,4076.0 ,4078.0 ,4079.0 ,4080.0 ,4081.0 ,4084.0 ,4085.0 ,4086.0 ,4087.0 ,4088.0 ,4089.0 ,4092.0 ,4094.0 ,4095.0 ,4096.0 ,4098.0 ,4100.0 ,4101.0 ,4102.0 ,4106.0 ,4107.0 ,4108.0 ,4112.0 ,4113.0 ,4114.0 ,4115.0 ,4119.0 ,4121.0 ,4123.0 ,4125.0 ,4126.0 ,4127.0 ,4128.0 ,4129.0 ,4131.0 ,4132.0 ,4255.0 ,9991305.0 ,9991306.0 ,9991311.0 ,9991312.0 ,9991313.0 ,9991314.0 ,9991316.0 ,9991324.0 ,9991325.0 ,9991661.0 ,9991662.0 ,9991663.0 ,9991666.0 ,9991669.0 ,9991670.0 ,9991671.0 ,9991673.0 ,9991674.0 ,9991675.0 ,9991676.0 ,9991677.0 ,9991679.0 ,9991680.0 ,9991681.0 ,9991682.0 ,9991683.0 ,9991684.0 ,9991685.0 ,9991686.0])

In [34]:
rasch_1 = rasch_1.drop(columns=[9991687.0 ,9991688.0 ,9991689.0 ,9991690.0 ,9991691.0 ,9991692.0 ,9991693.0 ,9991694.0 ,9991699.0 ,9991701.0 ,9991702.0 ,9991703.0 ,9991704.0 ,9991705.0 ,9991706.0 ,9991707.0 ,9991709.0 ,9991710.0 ,9991711.0 ,9991712.0 ,9991713.0 ,9991714.0 ,9991715.0 ,9991716.0 ,9991717.0 ,9991718.0 ,9991719.0 ,9991720.0 ,9991751.0 ,9991752.0 ,9991753.0 ,9991754.0 ,9991755.0 ,9991756.0 ,9991758.0 ,9991759.0 ,9991762.0 ,9991763.0 ,9991765.0 ,9991766.0 ,9991767.0 ,9991768.0 ,9991769.0 ,9991771.0 ,9991775.0 ,9991776.0 ,9991777.0 ,9991778.0 ,9991779.0 ,9991780.0 ,9991783.0 ,9991787.0 ,9991796.0 ,9991797.0 ,9991798.0 ,9991800.0 ,9991801.0 ,9991803.0 ,9991804.0 ,9991805.0 ,9991807.0 ,9991844.0 ,9991845.0 ,9991846.0 ,9991857.0 ,9991858.0 ,9991866.0 ,9991867.0 ,9991871.0 ,9991872.0 ,9991873.0 ,9991874.0 ,9991876.0 ,9991877.0 ,9991878.0 ,9991879.0 ,9991880.0 ,9991881.0 ,9991882.0 ,9991883.0 ,9991884.0 ,9991885.0 ,9991886.0 ,9991887.0 ])

In [43]:
rasch_1 = rasch_1.drop(columns=[9991888.0 ,9991890.0 ,9991892.0 ,9991893.0 ,9991894.0 ,9991896.0 ,9991897.0 ,9991898.0 ,9991899.0 ,9991900.0 ,9992081.0 ,9992082.0 ,9992083.0 ,9992084.0 ,9992085.0 ,9992087.0 ,9992090.0 ,9992091.0 ,9992092.0 ,9992093.0 ,9992095.0 ,9992096.0 ,9992097.0 ,9992098.0 ,9992099.0 ,9992100.0 ,9992101.0 ,9992102.0 ,9992103.0 ,9992104.0 ,9992105.0 ,9992106.0 ,9992107.0 ,9992108.0 ,9992109.0 ,9992201.0 ,9992204.0 ,9992205.0 ,9992217.0 ,9992218.0 ,9992228.0 ,9992231.0 ,9992232.0 ,9992233.0 ,9992234.0 ,9992235.0 ,9992236.0 ,9992237.0 ,9992238.0 ,9992239.0 ,9992240.0 ,9992241.0 ,9992242.0 ,9992243.0 ,9992244.0 ,9992245.0 ,9992248.0 ,9992249.0 ,9992250.0 ,9992251.0 ,9992253.0 ,9992254.0 ,9992255.0 ,9992256.0 ,9992257.0 ,9992258.0 ,9992259.0 ,9992260.0 ,9992262.0 ,9992263.0 ,9992264.0 ,9992265.0 ,9992266.0 ,9992270.0 ,9992271.0 ,9992276.0 ,9992279.0 ,9992280.0 ,9992281.0 ,9992283.0 ,9992289.0 ,9992290.0 ,9992359.0 ,9992360.0])

In [46]:
rasch_1 = rasch_1.drop(columns=[9992202.0 ,9992527.0 ,9992528.0 ,9992532.0 ,9992533.0 ,9992535.0 ,9992537.0 ,9992539.0 ,9992540.0 ,9992541.0 ,9992972.0 ,9992973.0 ,9992974.0 ,9992978.0 ,9992981.0 ,9992982.0 ,9992983.0 ,9992984.0 ,9992985.0 ,9992986.0 ,9992987.0 ,9992989.0 ,9992990.0 ,9992991.0 ,9992992.0 ,9992993.0 ,9992994.0 ,9992995.0 ,9992996.0 ,9992997.0 ,9992998.0 ,9992999.0 ,9993000.0 ,9993001.0])

In [49]:
rasch_1 = rasch_1.drop(columns=[137.0 ,812.0 ,9991781.0 ,9991784.0 ,9991790.0 ,9991802.0 ,9991806.0 ,9992252.0])

In [50]:
delete_li = []
for index, row in rasch_1.iterrows():
    cnt = row.count()
    if cnt < 2:
        delete_li.append(index)
        rasch_1 = rasch_1.drop([index])

rasch_1

SatzID,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,138.0,...,9992531.0,9992534.0,9992536.0,9992538.0,9992975.0,9992976.0,9992977.0,9992979.0,9992980.0,9992988.0
UserID,,,,,,,,,,,,,,,,,,,,,
499291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
rasch_1.to_csv('learning_data_period_1.csv')

In [52]:
df = pd.read_csv( 'learning_data_period_1.csv')
# user id as index
df=df.set_index('UserID')
# drop rows with 0 values
df = df.dropna(how='all')
# create index with four digits
df.index = df.index.astype(str).str.pad(7, side='left', fillchar='0')
# change 1.0 to 1
df = df.replace([1.0],'1')
# change 0.0 to 0
df = df.replace([0.0],'0')
# change NaN to .
df = df.fillna('.', inplace=False)

In [53]:
df.to_csv('concquest_input_data_period_1.csv')

In [ ]:
######## Manually
# open as csv in excel
## drop column row 
## row to string -> komma entfernen
## close csv
# open csv in editor
## save as dat

Time period 2

In [41]:
rasch_2 = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]
rasch_2['UserID'].astype(str)
rasch_2['SatzID'].astype(str)

# #es werden nur user aus der liste example user behalten
rasch_2 = pd.merge(example_user, rasch_2, on='UserID', how='left')

# only use input from the first time
rasch_2 = rasch_2[(rasch_2['Datum'] > '2022-11-21') & (rasch_2['Datum'] < '2023-02-21')]

In [42]:
test = rasch_2.groupby(['UserID','SatzID']).agg({'Datum':np.max}) # hat nichts mit den users zu tun, holt nur alle satz ids
test = test.reset_index()
test = pd.merge(test, rasch_2, on=['UserID','SatzID','Datum'])

In [43]:
rasch_2 = test.pivot_table(values='Erfolg', index=test.UserID, columns='SatzID', aggfunc='last')
rasch_2

SatzID,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,...,9992992.0,9992993.0,9992994.0,9992995.0,9992996.0,9992997.0,9992998.0,9992999.0,9993000.0,9993001.0
UserID,,,,,,,,,,,,,,,,,,,,,
499291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042810,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,...,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0
1042963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0


In [19]:
delete_li = []
for col in rasch_2:
    cnt = rasch_2[col].count()
    if cnt < 2:
        delete_li.append(col)
        

In [51]:
# rasch_2 = rasch_2.drop(columns=[127.0 ,128.0 ,129.0 ,131.0 ,135.0 ,137.0 ,145.0 ,147.0 ,149.0 ,151.0 ,154.0 ,155.0 ,223.0 ,224.0 ,225.0 ,227.0 ,230.0 ,231.0 ,234.0 ,235.0 ,238.0 ,241.0 ,245.0 ,246.0 ,247.0 ,248.0 ,251.0 ,253.0 ,255.0 ,258.0 ,259.0 ,260.0 ,261.0 ,266.0 ,268.0 ,269.0 ,270.0 ,272.0 ,273.0 ,274.0 ,276.0 ,373.0 ,374.0 ,375.0 ,376.0 ,378.0 ,379.0 ,380.0 ,381.0 ,382.0 ,383.0 ,384.0 ,385.0 ,386.0 ,387.0 ,391.0 ,392.0 ,393.0 ,394.0 ,395.0 ,396.0 ,397.0 ,398.0 ,399.0 ,400.0 ,401.0 ,583.0 ,584.0 ,587.0 ,589.0 ,591.0 ,592.0 ,598.0 ,600.0 ,601.0 ,608.0 ,610.0 ,643.0 ,647.0 ,648.0 ,650.0 ,653.0 ,654.0 ,657.0 ,661.0 ,664.0 ,665.0 ,666.0 ,667.0 ,668.0 ,670.0 ,671.0 ,672.0 ,779.0 ,781.0 ,782.0 ,790.0 ,791.0 ,794.0 ,797.0 ,800.0 ,803.0 ,806.0 ,808.0 ,814.0 ,816.0 ,830.0 ,831.0 ,832.0 ,833.0 ,834.0 ,837.0 ,869.0 ,872.0 ,874.0 ,875.0 ,879.0 ,880.0 ,881.0 ,882.0 ,883.0 ,886.0 ,887.0 ,889.0 ,890.0 ,891.0 ,893.0 ,894.0 ,895.0 ,896.0 ,897.0 ,1694.0])
# rasch_2 = rasch_2.drop(columns=[3905.0 ,3906.0 ,3907.0 ,3908.0 ,3909.0 ,3910.0 ,3911.0 ,3912.0 ,3913.0 ,3914.0 ,3915.0 ,3916.0 ,3917.0 ,3918.0 ,3919.0 ,3920.0 ,3921.0 ,3922.0 ,3923.0 ,3924.0 ,3925.0 ,3926.0 ,3927.0 ,3928.0 ,3929.0 ,3930.0 ,3931.0 ,3932.0 ,3933.0 ,3934.0 ,3936.0 ,3937.0 ,3939.0 ,3940.0 ,3941.0 ,3945.0 ,3947.0 ,3948.0 ,3950.0 ,3951.0 ,3952.0 ,3956.0 ,3960.0 ,3964.0 ,3966.0 ,3968.0 ,3971.0 ,3972.0 ,3975.0 ,3978.0 ,3980.0 ,3981.0 ,4073.0 ,4074.0 ,4075.0 ,4076.0 ,4077.0 ,4080.0 ,4081.0 ,4085.0 ,4087.0 ,4088.0 ,4089.0 ,4093.0 ,4094.0 ,4097.0 ,4098.0 ,4099.0 ,4100.0 ,4101.0])
# rasch_2 = rasch_2.drop(columns=[4128.0 ,4129.0 ,4255.0 ,9991301.0 ,9991304.0 ,9991305.0 ,9991306.0 ,9991311.0 ,9991312.0 ,9991314.0 ,9991316.0 ,9991317.0 ,9991320.0 ,9991321.0 ,9991322.0 ,9991323.0 ,9991325.0 ,9991326.0 ,9991328.0 ,9991661.0 ,9991662.0 ,9991663.0 ,9991666.0 ,9991669.0 ,9991670.0 ,9991672.0 ,9991673.0 ,9991674.0 ,9991675.0 ,9991679.0 ,9991680.0 ,9991681.0 ,9991682.0 ,9991683.0 ,9991684.0 ,9991685.0 ,9991687.0 ,9991688.0 ,9991689.0 ,9991690.0 ,9991691.0])
# rasch_2 = rasch_2.drop(columns=[1695.0 ,1696.0 ,1698.0 ,1699.0 ,1700.0 ,1705.0 ,1706.0 ,1707.0 ,1710.0 ,1711.0 ,1712.0 ,1713.0 ,1716.0 ,1717.0 ,2685.0 ,2686.0 ,2687.0 ,2691.0 ,2692.0 ,2693.0 ,2694.0 ,2695.0 ,2696.0 ,2698.0 ,2701.0 ,2702.0 ,2704.0 ,2705.0 ,2707.0 ,2709.0 ,2710.0 ,2711.0 ,2713.0 ,2714.0 ,3893.0 ,3894.0 ,3895.0 ,3896.0 ,3897.0 ,3898.0 ,3899.0 ,3900.0 ,3901.0 ,3902.0 ,3903.0 ,3904.0 ,4102.0 ,4103.0 ,4105.0 ,4107.0 ,4108.0 ,4109.0 ,4110.0 ,4115.0 ,4116.0 ,4117.0 ,4118.0 ,4119.0 ,4122.0 ,4125.0 ,4127.0 ,9991693.0 ,9991694.0 ,9991698.0 ,9991699.0 ,9991700.0 ,9991701.0 ,9991702.0 ,9991703.0 ,9991704.0 ,9991705.0 ,9991706.0 ,9991707.0 ,9991708.0 ,9991709.0 ,9991710.0 ,9991711.0 ,9991712.0 ,9991713.0 ,9991714.0 ,9991716.0 ,9991717.0 ,9991718.0 ,9991719.0 ,9991720.0 ,9991751.0 ,9991752.0 ,9991753.0 ,9991754.0 ,9991755.0 ,9991758.0 ,9991759.0 ,9991760.0 ,9991761.0 ,9991762.0 ,9991763.0 ,9991764.0 ,9991765.0 ,9991772.0 ,9991773.0 ,9991775.0])
# rasch_2 = rasch_2.drop(columns=[9991776.0 ,9991777.0 ,9991778.0 ,9991780.0 ,9991782.0 ,9991784.0 ,9991786.0 ,9991789.0 ,9991790.0 ,9991791.0 ,9991792.0 ,9991793.0 ,9991795.0 ,9991796.0 ,9991798.0 ,9991802.0 ,9991803.0 ,9991804.0 ,9991805.0 ,9991806.0 ,9991807.0 ,9991808.0 ,9991809.0 ,9991810.0 ,9991842.0 ,9991844.0 ,9991845.0 ,9991846.0 ,9991848.0 ,9991849.0 ,9991850.0 ,9991852.0 ,9991853.0 ,9991856.0 ,9991857.0 ,9991858.0 ,9991859.0 ,9991860.0 ,9991861.0 ,9991865.0 ,9991866.0 ,9991867.0 ,9991868.0 ,9991869.0 ,9991877.0 ,9991879.0 ,9991880.0 ,9991881.0 ,9991884.0 ,9991885.0 ,9991886.0 ,9991888.0 ,9991889.0 ,9991890.0 ,9991891.0 ,9991892.0 ,9991893.0 ,9991894.0 ,9991895.0 ,9991898.0 ,9991899.0 ,9992081.0 ,9992082.0 ,9992083.0 ,9992084.0 ,9992085.0 ,9992086.0 ,9992088.0 ,9992089.0 ,9992090.0 ,9992091.0 ,9992092.0 ,9992093.0 ,9992094.0 ,9992095.0 ,9992096.0 ,9992097.0 ,9992098.0 ,9992099.0 ,9992100.0 ,9992101.0 ,9992102.0 ,9992103.0 ,9992104.0])
# rasch_2 = rasch_2.drop(columns=[9992105.0 ,9992106.0 ,9992107.0 ,9992108.0 ,9992109.0 ,9992110.0 ,9992201.0 ,9992202.0 ,9992203.0 ,9992206.0 ,9992208.0 ,9992210.0 ,9992211.0 ,9992212.0 ,9992216.0 ,9992218.0 ,9992223.0 ,9992225.0 ,9992226.0 ,9992227.0 ,9992228.0 ,9992229.0 ,9992230.0 ,9992231.0 ,9992232.0 ,9992233.0 ,9992235.0 ,9992236.0 ,9992237.0 ,9992238.0 ,9992239.0 ,9992240.0 ,9992241.0 ,9992242.0 ,9992243.0 ,9992244.0 ,9992248.0 ,9992249.0 ,9992250.0 ,9992253.0 ,9992255.0 ,9992256.0 ,9992257.0 ,9992258.0 ,9992259.0 ,9992260.0 ,9992261.0 ,9992262.0 ,9992264.0 ,9992266.0 ,9992267.0 ,9992268.0 ,9992269.0 ,9992271.0 ,9992274.0 ,9992275.0 ,9992276.0 ,9992277.0 ,9992278.0 ,9992279.0 ,9992280.0 ,9992281.0 ,9992282.0 ,9992283.0 ,9992284.0 ,9992285.0 ,9992287.0 ,9992359.0 ,9992512.0 ,9992513.0 ,9992518.0 ,9992522.0 ,9992523.0 ,9992525.0 ,9992528.0 ,9992530.0 ,9992531.0 ,9992532.0 ,9992533.0 ,9992534.0 ,9992535.0 ,9992536.0 ,9992537.0 ,9992538.0])
#rasch_2 = rasch_2.drop(columns=[9992539.0 ,9992972.0 ,9992973.0 ,9992974.0 ,9992977.0 ,9992978.0 ,9992979.0 ,9992980.0 ,9992981.0 ,9992982.0 ,9992983.0 ,9992984.0 ,9992985.0 ,9992986.0 ,9992987.0 ,9992988.0 ,9992989.0 ,9992990.0 ,9992992.0 ,9992993.0 ,9992994.0 ,9992995.0 ,9992996.0 ,9992997.0 ,9992998.0 ,9992999.0 ,9993000.0 ,9993001.0])
rasch_2 = rasch_2.drop(columns=[134.0 ,141.0 ,144.0 ,3959.0 ,9991695.0 ,9991697.0 ,9991783.0 ,9991871.0 ,9991872.0 ,9991874.0 ,9991875.0 ,9991876.0 ,9991878.0 ,9991887.0 ,9991900.0 ,9992087.0])

In [55]:
delete_li = []
for index, row in rasch_2.iterrows():
    cnt = row.count()
    if cnt < 2:
        delete_li.append(index)
        rasch_2 = rasch_2.drop([index])

rasch_2

SatzID,130.0,132.0,133.0,136.0,138.0,139.0,140.0,142.0,143.0,146.0,...,9992521.0,9992524.0,9992526.0,9992527.0,9992529.0,9992540.0,9992541.0,9992975.0,9992976.0,9992991.0
UserID,,,,,,,,,,,,,,,,,,,,,
499291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042810,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
1042963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
rasch_2.to_csv('learning_data_period_2.csv')

In [54]:
df = pd.read_csv( 'learning_data_period_2.csv')
# user id as index
df=df.set_index('UserID')
# drop rows with 0 values
df = df.dropna(how='all')
# create index with four digits
df.index = df.index.astype(str).str.pad(7, side='left', fillchar='0')
# change 1.0 to 1
df = df.replace([1.0],'1')
# change 0.0 to 0
df = df.replace([0.0],'0')
# change NaN to .
df = df.fillna('.', inplace=False)
df.to_csv('concquest_input_data_period_2.csv')